In [1]:
import sys

In [2]:
# !{sys.executable} -m pip install --upgrade --user kfp==1.8.22

In [3]:
import kfp
from kfp import dsl
from functools import partial
from kfp.dsl import (
    pipeline,
    ContainerOp,
    PipelineVolume
)
from kfp.components import (
    InputPath,
    OutputPath,
    create_component_from_func
)

EXPERIMENT_NAME = 'llm' # Name of the experiment in the KF webapp UI
EXPERIMENT_DESC = 'llm med report pipeline experiment'

In [1]:
from dataclasses import dataclass

@dataclass
class Settings():
    llm_base_image: str = 'pytorch/pytorch:2.2.0-cuda11.8-cudnn8-devel'
    applyllm_version: str = '0.0.2'
    pypdf_version: str = '3.15.5'
    accelerate_version: str = '0.26.1'

settings = Settings()

In [5]:
from functools import partial

@partial(
    create_component_from_func,
    output_component_file=f"custom_registry_component.yaml",
    base_image=settings.llm_base_image, 
    packages_to_install=[
        f"applyllm=={settings.applyllm_version}",
        f"pypdf=={settings.pypdf_version}",
        f"accelerate=={settings.accelerate_version}"
    ], # adding additional libs
    # pip_index_urls=["https://gitlab.lrz.de/api/v4/projects/150553/packages/pypi/simple"]
    # define my private pypi package registry v2 component decorator
)
def llm_op(model_root: str):
    import os
    import applyllm as apl

    from applyllm.accelerators import (
        AcceleratorHelper,
        AcceleratorStatus,
        DirectorySetting,
        # DIR_MODE_MAP,
        TokenHelper as th
    )
    from applyllm.utils import time_func
    from applyllm.pipelines import (
        LocalCausalLMConfig,
        ModelConfig,
        ModelCatalog,
    )
    import transformers
    import torch
    from transformers import AutoModelForCausalLM, AutoTokenizer
    from torch import bfloat16


    print(f"applyllm version:     {apl.__version__}")
    print(f"transformers version: {transformers.__version__}")
    print(f"torch version:        {torch.__version__}")

    dir_setting = DirectorySetting(home_dir=model_root)

    gpu_status = AcceleratorStatus.create_accelerator_status()
    '''init gpu helper'''
    gpu_helper = AcceleratorHelper()
    UUIDs = gpu_helper.nvidia_device_uuids_filtered_by(is_mig=True, log_output=False)
    gpu_helper.init_cuda_torch(UUIDs, dir_setting)
    print(os.environ["CUDA_VISIBLE_DEVICES"])
    print(os.environ["XDG_CACHE_HOME"])
    '''init llm model to be loaded'''
    model_map = {
        "llama7B-chat":     "meta-llama/Llama-2-7b-chat-hf",
        "llama13B-chat" :   "meta-llama/Llama-2-13b-chat-hf",
        "llama70B-chat" :   "meta-llama/Llama-2-70b-chat-hf",
        "mistral7B-01":     "mistralai/Mistral-7B-v0.1",
        "mistral7B-inst02": "mistralai/Mistral-7B-Instruct-v0.2",
        "mixtral8x7B-01":   "mistralai/Mixtral-8x7B-v0.1",
        "mixtral8x7B-inst01":   "mistralai/Mixtral-8x7B-Instruct-v0.1", 
    }
    model_type = "mistral7B-inst02"
    model_name = model_map.get(model_type, "7B")
    print(model_name)

    token_kwargs = th.gen_token_kwargs(model_type=model_type, dir_setting=dir_setting)

    base_lm_config = ModelConfig(
        model_config = {
            "pretrained_model_name_or_path": model_name,
            "device_map": "auto",
            # "max_memory": f"{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB",
        }
    )

    lm_model_kwargs = {
        "quantized": True,
        "model_config": base_lm_config.get_config(),
        "quantization_config": {
            "quantization_config": transformers.BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type='nf4',
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=bfloat16
            )
        }
    }
    lm_config = LocalCausalLMConfig(**lm_model_kwargs)

    @time_func
    def fetch_lm_model():
        return AutoModelForCausalLM.from_pretrained(
        **lm_config.get_config(),
        **token_kwargs,  
        )

    model = fetch_lm_model()

    gpu_status.gpu_usage()


In [6]:
def set_res_limits(task: ContainerOp, mem_req="200Mi", cpu_req="2000m", mem_lim="4000Mi", cpu_lim='4000m', gpu_req=None, gpu_lim=None, gpu_type:str="20gb"):
    """
    this function helps to set the resource limit for container operators
    op.set_memory_limit('1000Mi') = 1GB
    op.set_cpu_limit('1000m') = 1 cpu core
    """
    if gpu_type == "20gb":
        gpu_resource = "nvidia.com/mig-2g.20gb"
        # gpu_resource = "nvidia.com/mig-1g.20gb"
    else:
        gpu_resource = "nvidia.com/mig-1g.10gb"
        
    # gpu_resource = "nvidia.com/mig-2g.20gb"
    new_op = task.set_memory_request(mem_req)\
        .set_memory_limit(mem_lim)\
        .set_cpu_request(cpu_req)\
        .set_cpu_limit(cpu_lim)
    if (gpu_req is not None) and (gpu_lim is not None):
        new_op.add_resource_request(gpu_resource, gpu_req)
        new_op.add_resource_limit(gpu_resource, gpu_lim)
    return new_op

In [7]:
@pipeline(
    name = EXPERIMENT_NAME,
    description = EXPERIMENT_DESC
)
def llm_pipeline(model_root: str = "/mnt"):
    '''local variable'''
    no_artifact_cache = "P0D"
    artifact_cache_today = "P1D"
    # cache_setting = artifact_cache_today
    cache_setting = no_artifact_cache

    '''Pipeline Volume'''
    shared_volume = PipelineVolume("llm-models")
    
    '''pipeline'''   
    llm_task = llm_op(model_root=model_root)
    # 200 MB ram and 1 cpu
    llm_task = set_res_limits(task=llm_task, mem_req="20Gi", mem_lim="40Gi",
                            cpu_req="2000m", cpu_lim="10000m", 
                            gpu_req=1, gpu_lim=1, gpu_type="20gb")
    # set the download caching to be 1day, disable caching with P0D
    # download_task.execution_options.caching_strategy.max_cache_staleness = artifact_cache_today
    llm_task.add_pvolumes({model_root: shared_volume})
    llm_task.execution_options.caching_strategy.max_cache_staleness = cache_setting
    llm_task.set_display_name("llm op")

In [8]:
import os
pipeline_path_dir="./compiled"
if not os.path.exists(pipeline_path_dir):
    os.makedirs(pipeline_path_dir)

PIPE_LINE_FILE_NAME=f"llm_rag_kfp_pipeline"
kfp.compiler.Compiler().compile(llm_pipeline, f"{pipeline_path_dir}/{PIPE_LINE_FILE_NAME}.yaml")

In [9]:
from datetime import datetime
from pytz import timezone as ptimezone

def get_local_time_str(target_tz_str: str = "Europe/Berlin", format_str: str = "%Y-%m-%d %H-%M-%S") -> str:
    """
    this method is created since the local timezone is miss configured on the server
    @param: target timezone str default "Europe/Berlin"
    @param: "%Y-%m-%d %H-%M-%S" returns 2022-07-07 12-08-45
    """
    target_tz = ptimezone(target_tz_str) # create timezone, in python3.9 use standard lib ZoneInfo
    # utc_dt = datetime.now(datetime.timezone.utc)
    target_dt = datetime.now(target_tz)
    return datetime.strftime(target_dt, format_str)

In [10]:
# from kubernetes import client as k8s_client
pipeline_config = dsl.PipelineConf()

# pipeline_config.set_image_pull_secrets([k8s_client.V1ObjectReference(name=K8_GIT_SECRET_NAME, namespace=NAME_SPACE)])
# pipeline_config.set_image_pull_policy("Always")
pipeline_config.set_image_pull_policy("IfNotPresent")

pipeline_args = {
    "model_root": "/mnt"
}

In [11]:
RUN_NAME = f"{PIPE_LINE_FILE_NAME} {get_local_time_str()}"

client = kfp.Client()
NAMESPACE = client.get_user_namespace()

In [12]:
run = client.create_run_from_pipeline_func(
    pipeline_func=llm_pipeline,
    arguments = pipeline_args, #{}
    run_name = RUN_NAME,
    pipeline_conf=pipeline_config,
    experiment_name=EXPERIMENT_NAME,
    namespace=NAMESPACE,
)

run

RunPipelineResult(run_id=9405bb18-df37-408f-96f3-19fb50d0423b)